## Measuring Heart Rhythm Disorders

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import heartpy as hp
from numpy import savetxt

In [5]:
df=pd.read_csv("D:\Flipr Hackathon XIII ML Task\ECG_training.csv")

In [29]:
df

,ECG,Classification
0,"-127,-162,-197,-229,-245,-254,-261,-265,-268,-...",N
1,"128,157,189,226,250,257,262,265,268,269,268,26...",N
2,"519,619,723,827,914,956,955,934,920,900,889,88...",A
3,"-266,-316,-367,-407,-423,-423,-401,-367,-329,-...",N
4,"21,22,24,26,28,31,32,34,34,35,35,34,34,34,35,3...",N
...,...,...
3495,"347,378,410,540,695,736,751,778,849,803,688,67...",N
3496,"-59,-71,-87,-104,-111,-114,-117,-119,-120,-121...",N
3497,"-60,-83,-106,-128,-145,-155,-163,-168,-173,-17...",N
3498,"582,702,807,893,962,997,984,945,927,915,906,89...",N


In [15]:
df.columns

Index(['ECG', 'Classification'], dtype='object')

## Feature Extraction

In [53]:

total_rows=100


#create a dataframe and empty list
new_df=pd.DataFrame(columns=['bpm','ibi','sdnn','sdsd','rmssd','pnn20','pnn50','hr_mad','sd1','sd2','s','sd1/sd2','breathingrate'])
new_list=[]

#specify the no. of rows or data you need
for row_no in range(total_rows):              
    #convert each row(str format) to int
    d=df["ECG"][row_no].split(",")
    for i in range(0, len(d)): 
        d[i] = int(d[i])
    #convert the list to 1d or numpy array and save the file with respective row no.
    n=np.array(d)
    savetxt('array_N{}.csv'.format(row_no) , n , delimiter=',')
    
    
    #heartpy
    sample_rate = 300
    data = hp.get_data('array_N{}.csv'.format(row_no))

    wd, m = hp.process(data, sample_rate)

    filtered = hp.filter_signal(data, cutoff = 0.05, sample_rate = sample_rate, filtertype='notch')

    wd, m = hp.process(hp.scale_data(filtered), sample_rate)

    from scipy.signal import resample
    #resample the data. Usually 2, 4, or 6 times is enough depending on original sampling rate
    resampled_data = resample(filtered, len(filtered) * 2)
    #And run the analysis again. Don't forget to up the sample rate as well!
    wd, m = hp.process(hp.scale_data(resampled_data), sample_rate * 2)

    wd, m = hp.process(hp.scale_data(data), sample_rate)

    resampled_data = resample(data, len(filtered) * 2)
    #And run the analysis again. Don't forget to up the sample rate as well!
    wd, m = hp.process(hp.scale_data(resampled_data), sample_rate * 2)
    
    
    #get the measures from pacakge and store it as dataframe
    for measure in m.keys():
        new_list.append(m[measure])
    for j in range(len(new_df.columns)):
        new_df.loc[row_no,:]=new_list
        break
    new_list.clear()



In [54]:
new_df

,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,78.774617,761.666667,60.836037,82.802963,92.796882,0.513514,0.108108,29.166667,65.6148,56.871309,11723.165788,1.153742,0.241938
1,249.855408,240.138889,119.529282,103.390068,221.204615,1.0,1.0,74.166667,154.746906,163.696686,79581.429165,0.945327,0.347584
2,64.433649,931.190476,169.459843,133.948286,219.769207,0.923077,0.807692,135.833333,155.185966,183.420635,89423.262563,0.846066,0.19185
3,153.387303,391.166667,169.807162,144.939068,341.605686,1.0,1.0,125.0,238.283911,140.612549,105261.286312,1.694613,0.25582
4,62.666821,957.444444,51.791486,20.531001,29.724147,0.413793,0.103448,46.666667,20.789987,69.426715,4534.513473,0.299452,0.13931
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,197.769269,303.383838,180.252575,134.15414,299.106076,1.0,1.0,115.0,210.060805,141.765211,93554.474916,1.481751,0.09999
96,65.475071,916.37931,52.835444,43.4439,82.689177,0.821429,0.607143,28.333333,58.465169,47.432447,8712.095219,1.232599,0.338766
97,91.812081,653.508772,45.439359,41.355318,43.203152,0.055556,0.027778,22.5,30.2605,41.070738,3904.437605,0.73679,0.0
98,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
for k in range(total_rows):
    new_df['classification'][k]=df['Classification'][k]
new_df

,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,classification
0,78.774617,761.666667,60.836037,82.802963,92.796882,0.513514,0.108108,29.166667,65.6148,56.871309,11723.165788,1.153742,0.241938,N
1,249.855408,240.138889,119.529282,103.390068,221.204615,1.0,1.0,74.166667,154.746906,163.696686,79581.429165,0.945327,0.347584,N
2,64.433649,931.190476,169.459843,133.948286,219.769207,0.923077,0.807692,135.833333,155.185966,183.420635,89423.262563,0.846066,0.19185,A
3,153.387303,391.166667,169.807162,144.939068,341.605686,1.0,1.0,125.0,238.283911,140.612549,105261.286312,1.694613,0.25582,N
4,62.666821,957.444444,51.791486,20.531001,29.724147,0.413793,0.103448,46.666667,20.789987,69.426715,4534.513473,0.299452,0.13931,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,197.769269,303.383838,180.252575,134.15414,299.106076,1.0,1.0,115.0,210.060805,141.765211,93554.474916,1.481751,0.09999,N
96,65.475071,916.37931,52.835444,43.4439,82.689177,0.821429,0.607143,28.333333,58.465169,47.432447,8712.095219,1.232599,0.338766,N
97,91.812081,653.508772,45.439359,41.355318,43.203152,0.055556,0.027778,22.5,30.2605,41.070738,3904.437605,0.73679,0.0,N
98,NaN,NaN,NaN,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


## Save dataframe as csv

In [57]:
new_df.to_csv('Part1_reboot.csv')